<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2


    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.60


    Uninstalling yfinance-0.2.60:
      Successfully uninstalled yfinance-0.2.60
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-12 13:17:10
-------------------
qualified stocks: 96
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.58 L
Current:  1.26 C
-------------------
Today PnL: 4.35 L (3.58%)
Current PnL: -19.70 L (-13.97%)
CY Booked + Current PnL: -16.86 L (-11.96%)
-------------------
Total profit:  3.70 L
Total loss:  -23.39 L
-------------------
Total Booked + Current PnL: 10.12 L (8.87%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 82.39 L (65.43%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.44 C (114.48%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 7.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,21.99,X-MC,91.67,140000.0,25222.0,10500.0,1.69,21.98,7.50,31.13,215.0,2.40,1.13,36.37,AR,ATH,TYRES
71,SBILIFE,1496.49,1924.99,-1.16,H-LC,85.42,146773.0,21068.0,14927.0,2.79,16.76,10.17,28.63,116.0,1.41,1.18,30.76,AR,ATH,INSURANCE
58,PIDILITIND,2504.06,3576.00,-11.81,X-LC,47.92,91986.0,16864.0,15297.0,2.89,22.45,16.63,42.81,87.0,1.10,0.74,15.36,X40,BTT,CHEMICALS
72,SBIN,760.30,863.00,-15.73,M-LC,60.42,212066.0,10586.0,16626.0,2.69,5.25,7.84,13.51,91.0,0.64,1.71,16.18,XY25,NTT,BANKS
56,NMDC,65.62,82.00,34.50,M-MC,58.33,103780.0,3381.0,21680.0,5.51,3.37,20.89,24.96,73.0,0.16,0.83,16.97,XY25,NTT,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,8421.60,10420.45,111.67,X-SC,94.79,84695.0,-24786.0,50775.0,11.71,-22.64,59.95,23.73,122.0,-0.49,0.68,28.19,X40N,ATH,CERAMICS
90,VALIANTORG,512.64,838.00,-294.01,H-SC,1.04,82446.0,-50840.0,135434.0,9.99,-38.14,164.27,63.47,83.0,-0.38,0.66,38.20,XR,NTT,CHEMICALS
16,BSOFT,628.85,844.81,-22.54,M-SC,84.38,72818.0,-37231.0,75024.0,8.23,-33.83,103.03,34.34,20.0,-0.50,0.59,19.88,XR,ATH,IT
84,TITAGARH,1097.23,1548.00,-25.94,H-SC,7.29,129964.0,-62051.0,140933.0,8.06,-32.32,108.44,41.08,76.0,-0.44,1.05,10.00,XY24,NTT,ENGINEERING
67,SAIL,130.64,228.00,77.92,M-MC,56.25,202266.0,-22696.0,190353.0,7.75,-10.09,94.11,74.53,217.0,-0.12,1.63,17.44,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1354.50,1800.0,-37.01,H-MC,77.08,46637.0,-30570.0,55964.0,-1.04,-39.59,120.00,32.89,41.0,-0.55,0.38,28.44,XR,NTT,BANKS
12,BANKINDIA,113.49,190.0,-28.40,M-MC,25.00,124600.0,-4552.0,91618.0,-0.66,-3.52,73.53,67.42,71.0,-0.05,1.00,21.18,XR,NTT,BANKS
57,PFIZER,4236.33,6318.3,-15.37,H-SC,40.62,132014.0,688.0,63855.0,0.20,0.52,48.37,49.15,102.0,0.01,1.06,8.81,X40,ATH,PHARMA
69,SATIN,199.77,274.0,-15.90,H-SC,80.21,165975.0,-29999.0,102822.0,0.59,-15.31,61.95,37.16,142.0,-0.29,1.33,23.95,XY24,NTT,FINANCE
45,KANSAINER,299.63,340.0,-66.58,H-SC,42.71,233955.0,-35712.0,72035.0,0.85,-13.24,30.79,13.47,167.0,-0.50,1.88,17.10,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TATAMOTORS,776.02,1065.00,-52.64,M-LC,95.83,190495.0,-15150.0,91723.0,1.46,-7.37,48.15,37.24,1.0,-0.17,1.53,23.99,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.80,H-LC,73.96,139000.0,6016.0,84832.0,2.20,4.52,61.03,68.31,2.0,0.07,1.12,13.34,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-27.09,H-LC,39.58,121005.0,-1998.0,64701.0,6.42,-1.62,53.47,50.98,3.0,-0.03,0.97,10.56,AR,ATH,REALTY
10,AWL,342.88,485.00,-69.22,H-MC,6.25,167100.0,-50629.0,140882.0,4.18,-23.25,84.31,41.45,5.0,-0.36,1.34,11.62,XY24,NTT,FMCG
94,WHIRLPOOL,1151.42,2270.00,-35.80,M-SC,93.75,161760.0,15530.0,126529.0,4.15,10.62,78.22,97.15,7.0,0.12,1.30,38.19,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,21.99,X-MC,91.67,140000.0,25222.0,10500.0,1.69,21.98,7.50,31.13,215.0,2.40,1.13,36.37,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.54,186838.0,31403.0,39030.0,5.02,20.20,20.89,45.32,198.0,0.80,1.50,27.13,XY24,NTT,HEALTHCARE
58,PIDILITIND,2504.06,3576.00,-11.81,X-LC,47.92,91986.0,16864.0,15297.0,2.89,22.45,16.63,42.81,87.0,1.10,0.74,15.36,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,86.46,186936.0,35730.0,29928.0,4.46,23.63,16.01,43.43,37.0,1.19,1.50,29.58,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NHPC,82.08,115.00,192.82,M-MC,14.58,141821.0,808.0,55750.0,5.87,0.57,39.31,40.11,212.0,0.01,1.14,15.94,XY24,NTT,POWER
56,NMDC,65.62,82.00,34.50,M-MC,58.33,103780.0,3381.0,21680.0,5.51,3.37,20.89,24.96,73.0,0.16,0.83,16.97,XY25,NTT,MINING
59,POLYCAB,5005.92,7541.35,17.52,H-MC,90.62,166936.0,26770.0,44221.0,3.39,19.10,26.49,50.65,54.0,0.61,1.34,27.41,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,86.46,186936.0,35730.0,29928.0,4.46,23.63,16.01,43.43,37.0,1.19,1.50,29.58,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.00,-2.28,H-SC,67.71,159204.0,7840.0,37938.0,5.28,5.18,23.83,30.25,131.0,0.21,1.28,26.01,X40N,NTT,FINANCE


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
95,WIPRO,243.46,420.00,-7.44,M-LC,61.46,158763.0,7818.0,101640.0,5.81,5.18,64.02,72.51,58.0,0.08,1.28,19.54,XR,NTT,IT
18,CAMS,3643.00,5250.99,-7.21,H-SC,9.38,102721.0,717.0,44304.0,5.43,0.70,43.13,44.14,47.0,0.02,0.83,18.99,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-53.52,H-SC,76.04,100434.0,-1293.0,115670.0,1.20,-1.27,115.17,112.43,NaN,-0.01,0.81,18.54,AR,ATH,MISC
93,VOLTAS,1272.00,1929.20,-20.25,H-MC,23.96,135916.0,-4004.0,76290.0,1.02,-2.86,56.13,51.67,19.0,-0.05,1.09,2.66,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-28.40,M-MC,25.00,124600.0,-4552.0,91618.0,-0.66,-3.52,73.53,67.42,71.0,-0.05,1.00,21.18,XR,NTT,BANKS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,60.00,-41.46,M-SC,75.00,114264.0,-28215.0,47739.0,3.90,-19.80,41.78,13.70,219.0,-0.59,0.92,34.86,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,21.99,X-MC,91.67,140000.0,25222.0,10500.0,1.69,21.98,7.50,31.13,215.0,2.40,1.13,36.37,AR,ATH,TYRES
88,UNIONBANK,123.87,163.00,-8.12,M-LC,59.38,145206.0,4366.0,40120.0,3.91,3.10,27.63,31.59,83.0,0.11,1.17,26.13,XY24,NTT,BANKS
71,SBILIFE,1496.49,1924.99,-1.16,H-LC,85.42,146773.0,21068.0,14927.0,2.79,16.76,10.17,28.63,116.0,1.41,1.18,30.76,AR,ATH,INSURANCE
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.54,186838.0,31403.0,39030.0,5.02,20.20,20.89,45.32,198.0,0.80,1.50,27.13,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.0,-799.62,H-SC,100.00,66661.0,-28655.0,73347.0,3.45,-30.06,110.03,46.89,51.0,-0.39,0.54,31.48,X40N,NTT,MISC
81,TATAELXSI,7332.28,9000.0,-19.51,X-MC,98.96,84539.0,-18113.0,41458.0,5.24,-17.64,49.04,22.74,33.0,-0.44,0.68,27.07,X40N,NTT,IT
60,QUESS,711.66,986.0,NaN,NaN,97.92,77559.0,-78295.0,138373.0,7.58,-50.24,178.41,38.55,NaN,-0.57,0.62,18.76,XY24,NTT,MISC
1,AARTIIND,487.04,919.0,22.69,M-SC,96.88,77647.0,-3689.0,75830.0,2.96,-4.54,97.66,88.69,118.0,-0.05,0.62,30.93,XR,NTT,CHEMICALS
82,TATAMOTORS,776.02,1065.0,-52.64,M-LC,95.83,190495.0,-15150.0,91723.0,1.46,-7.37,48.15,37.24,1.0,-0.17,1.53,23.99,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.13
1,25,40.51
2,50,68.40


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.11
X40     14.36
X40N    13.71
AR      13.50
XY25    10.96
XR      10.10
X200     1.18
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.66
X-LC    18.16
M-SC    16.15
M-LC     8.31
M-MC     7.35
X-MC     6.69
H-LC     6.64
H-MC     6.62
X-SC     2.77
L-SC     2.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.68,-2.20,34.71
BANKS,10.84,-11.84,66.69
IT,9.15,-21.79,89.73
FINANCE,7.18,-23.89,79.79
MISC,7.03,-29.54,85.29
INSURANCE,5.90,-5.26,36.99
PAINTS,5.53,-19.63,55.03
ELECTRICAL,4.03,-11.16,67.80
REFINERIES,3.62,9.45,7.24


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3181534.0,28
XR,1289066.0,15
AR,1277431.0,15
X40N,1226449.0,17
X40,810227.0,12
SR,208699.0,2
XY25,174331.0,6
X200,71480.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2497153.0,22
M-SC,1996041.0,20
X-LC,726551.0,12
M-MC,675839.0,7
H-MC,404688.0,6
X-SC,364474.0,4
M-LC,347211.0,6
X-MC,324740.0,6
L-SC,322744.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947803.0      6
M-SC       XY24      798623.0      6
H-SC       X40N      572378.0      7
           AR        559719.0      3
M-MC       XY24      476528.0      4
M-SC       XR        437820.0      5
           X40N      380375.0      5
X-LC       X40       369464.0      5
H-SC       XR        312714.0      4
M-LC       XY24      228945.0      4
M-SC       X40       200118.0      2
H-LC       AR        196613.0      4
X-LC       AR        188546.0      3
H-MC       XY24      184792.0      2
L-SC       XR        164887.0      2
X-MC       XY24      142073.0      2
X-SC       XR        124423.0      1
           SR        122290.0      1
           X40N      117761.0      2
X-MC       X40N      111714.0      2
H-SC       X40       104539.0      2
M-LC       XR        101640.0      1
X-LC       XY25       97061.0      3
M-SC       AR         92696.0      1
M-MC       XR         91618.0      1
L-SC       XY24       90803.0      1
M-SC       SR         86409.0      1
M-MC       AR         86013.0      1
H-MC       AR         76290.0      1
X-LC       X200       71480.0      1
L-SC       AR         67054.0      1
X-MC       X40        60453.0      1
H-MC       XR         55964.0      1
           X40N       44221.0      1
           X40        43421.0      1
H-LC       XY25       38964.0      1
           X40        32232.0      1
M-MC       XY25       21680.0      1
M-LC       XY25       16626.0      1
X-MC       AR         10500.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
